# Credmark Modeling Framework Example for Jupyter notebook
## Ledger Model

version: 2023.4.15

## Initialize

<div class="alert alert-block alert-info">
    <b>Note:</b> Change to a web3 provider you have in `params`/`chain_to_provider_url` below.
</div>

In [10]:
%reload_ext credmark.cmf.ipython

cmf_param = {
    'chain_id': 1,
    'block_number': None,
    # 'chain_to_provider_url': {'1': 'https://mainnet.infura.io/v3/... or https://eth-mainnet.g.alchemy.com/'}, # or, use the credmark's nodes
    'api_url': None,
    'use_local_models': None,
    'register_utility_global': True}

context, model_loader = %cmf cmf_param

## Example

### Contract - Uni V2

In [11]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
print(contract.functions.name().call(), contract.functions.symbol().call())

for addr in [contract.functions.token0().call(), contract.functions.token1().call()]:
    print(Token(address=addr).symbol)

Uniswap V2 UNI-V2
WETH
USDT


### Ledger Examples

- Get the max value group by `to`

In [12]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')

with contract.ledger.functions.Transfer as q:
    ret = (q.select(
        aggregates=[(q.FN_VALUE.max_(), 'max_value')],
        group_by=[q.FN_TO,q.BLOCK_NUMBER],
        order_by=q.field('max_value').dquote().desc(),
        where=q.BLOCK_NUMBER.gt(context.block_number - 2000000),
        limit=10
    ).to_dataframe())
display(ret)

""


- Get max amount0Out by `to`.

In [13]:
with contract.ledger.events.Swap as q:
    ret = (q.select(
        aggregates=[(q.EVT_AMOUNT0OUT.max_(), 'max_value')],
        group_by=[q.EVT_TO],
        order_by=q.field('max_value').dquote().desc(),
        where=q.BLOCK_NUMBER.gt(context.block_number - 2000000),
        limit=5)
    .to_dataframe())
display(ret)

,max_value,evt_to
0,1566237914550983720960.0,0x00000000003b3cc22af3ae1eac0440bcee416b40
1,1495609958420514340864.0,0x00000000008c4fb1c916e0c88fd4cc402d935e7d
2,973881389037030900000,0x11b815efb8f581194ae79006d24e0d814b7697f6
3,674617609601655600000,0xe8c060f8052e07423f71d445277c61ac5138a2e5
4,542876964979180900000,0xed77777586d73c58eb4d6bebdf9c85c2d5f56c2d


- Get max amount0Out and max amount1Out

In [14]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
with contract.ledger.events.Swap as q:
    ret = (q.select(
        aggregates=[(q.EVT_AMOUNT0OUT.max_(), 'max_amount0Out'),
                    (q.EVT_AMOUNT1OUT.max_(), 'max_amount1Out')]
    ).to_dataframe())
display(ret)

,max_amount0Out,max_amount1Out
0,9.609250e+22,50000000000000


- Get the transaction hash containing the max amount0Out.

In [15]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
with contract.ledger.events.Swap as q:
    ret2 = (q.select(
        aggregates=[(q.EVT_AMOUNT0OUT.max_(), 'max_amount0Out')],
        group_by=[q.TXN_HASH],
        having=q.EVT_AMOUNT0OUT.max_().ge(ret.max_amount0Out[0]),
    ).to_dataframe())
display(ret2)

,transaction_hash,max_amount0Out
0,0x8bb8dc5c7c830bac85fa48acad2505e9300a91c3ff23...,9.609250e+22


- Get the transaction hash containing the max amount0Out or max amount1Out

For any column in `having` needs to be in `aggregates`.

In [16]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
with contract.ledger.events.Swap as q:
    ret3 = (q.select(
        aggregates=[(q.EVT_AMOUNT0OUT.max_(), 'max_amount0Out'),
                    (q.EVT_AMOUNT1OUT.max_(), 'max_amount1Out')],
        group_by=[q.BLOCK_NUMBER, q.TXN_HASH],
        having=(
            q.EVT_AMOUNT0OUT.max_().ge(ret.max_amount0Out[0]).or_(
            q.EVT_AMOUNT1OUT.max_().ge(ret.max_amount1Out[0]))),
        order_by=q.BLOCK_NUMBER
    ).to_dataframe())

display(ret3)

,block_number,transaction_hash,max_amount0Out,max_amount1Out
0,11129500,0x0fc6d2ca064fc841bc9b1c1fad1fbb97bcea5c9a1b2b...,0.000000e+00,50000000000000
1,11129501,0xa9e80d1214eb369ca443f4a025d7605914f758bc8103...,0.000000e+00,50000000000000
2,11129502,0x789da3bb1e14a7ee5014e8150d7c28dc43bf40caa717...,0.000000e+00,50000000000000
3,11129503,0x7fa34152553f2ce7e6f22ca0ada802ab2c18e0ef9410...,0.000000e+00,50000000000000
4,11129504,0x89316cf5d24c82ca2dbbd48989d4c62e44bccaecdcf3...,0.000000e+00,50000000000000
5,11129505,0x0ab67369a6beb7875df6f2917916851530e9b10c46ea...,0.000000e+00,50000000000000
6,11129506,0x0f6dee0f75aa648c15f61bcc89ae86835a2c55fdd525...,0.000000e+00,50000000000000
7,11129507,0x8ed270d3370fbc4499b66178df7ca3fb00772e8e4734...,0.000000e+00,50000000000000
8,11129509,0xc2cc773009ebaee06c1301a485794cbf0a56e657ac30...,0.000000e+00,50000000000000
9,11129510,0xe60b28f063036c9055f1ea92f09e3467d6e02cb90226...,0.000000e+00,50000000000000


In [17]:
# Below web3 may not work with some node servers
if False:
    contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
    swap_events = contract.events.Swap.createFilter(
        fromBlock=0,
        toBlock=context.block_number
    ).get_all_entries()


In [18]:
10 / 100, block_number - 1000, block_number - 990

(0.1, 17063589, 17063599)

In [19]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
pd.DataFrame(contract.fetch_events(contract.events.Swap, from_block=block_number-1000, to_block=block_number-990, by_range=11))

,args,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber,sender,to,amount0In,amount1In,amount0Out,amount1Out
0,"(sender, to, amount0In, amount1In, amount0Out,...",Swap,120,48,"b""2\xa4\x92\xfd\x13}F\xcd\xea\xce\x9a\x84E'\x9...",0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852,"b""r\x9e\xafJp\x05*\x9b'\x11\x10\x94l$\xb1\xcb\...",17063589,0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45,0x48D20b3e529fB3DD7D91293f80638dF582AB2Daa,0,208000000,99189683032255386,0
1,"(sender, to, amount0In, amount1In, amount0Out,...",Swap,243,106,b'\x94%\xcfG\xe1\xad\x1d\x847\xc4\xf4.\xc9h\xa...,0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852,"b""r\x9e\xafJp\x05*\x9b'\x11\x10\x94l$\xb1\xcb\...",17063589,0xEf1c6E67703c7BD7107eed8303Fbe6EC2554BF6B,0x185CeA29D6Ac8aB1054080a6A4151541425CE474,0,2000000000,953679472755813693,0
2,"(sender, to, amount0In, amount1In, amount0Out,...",Swap,225,90,b'\x9b\xa7\xd4\x84\xcf7\xa9\x9fE\xc5\x91\xad\x...,0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852,"b""\x02\xdc\xd6\xa6g@z4\x02x;\xec\xe9 \x97\xa0\...",17063590,0xEf1c6E67703c7BD7107eed8303Fbe6EC2554BF6B,0x8AB4309019d7674C6112Eb32698A30fDCba6a278,0,1000000000,476793820127138586,0
3,"(sender, to, amount0In, amount1In, amount0Out,...",Swap,124,55,"b""l:\xb4\xc7\xe8p\xa5\rDx\x956;q\x06u\xae\xf4\...",0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852,"b'i<\xf2\xca_\xaf\xd3\r\x88,{.4\x9d\xa4\xa3\x1...",17063591,0xEf1c6E67703c7BD7107eed8303Fbe6EC2554BF6B,0xA66cdad5FAcbCBF74d9B60DCc8034c2A485e63Ff,0,100000000,47677697358988479,0
4,"(sender, to, amount0In, amount1In, amount0Out,...",Swap,18,2,b'\x8b\xf8\xae\\6QH\xb7\x10\x1b\x80\x01\xe4\x0...,0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852,"b'\xcd<\xd2\x1f\n"",\x06\x01h\xe6\xf5\x0bF\xacm...",17063593,0xEf1c6E67703c7BD7107eed8303Fbe6EC2554BF6B,0xA43fe16908251ee70EF74718545e4FE6C5cCEc9f,0,2500000000,1191843157792442609,0
5,"(sender, to, amount0In, amount1In, amount0Out,...",Swap,104,66,b'\x04=]\xc5>\x14+\x8cV!mN\x96y\x94\xfeZ\xb8|\...,0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852,b't1w\xf5\x1b6p/\xd7B\xc1ct6\xe8\xd6\x83j3Hjb\...,17063596,0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D,0xA6B59E01dd182ca0D255973bf8b3059225E57842,49577114399571467,0,0,103377471
